In [123]:
import os
import ast
import pandas as pd
import re

get files

In [124]:
file_list = {}

def list_files_in_directory(directory):
    for root, _, files in os.walk(directory):
        for file in files:
            if not "env" in root and not "test" in file and not "__init__" in file and file.endswith('.py'):
                file_list[os.path.join(root, file)] = []

list_files_in_directory("/home/dan/Documents/bnb_ai")

In [125]:
# import subprocess
# for codefile in file_list:
#     output = subprocess.run(f"pylint {codefile}", shell=True)
#     print(output.stdout)

get dependencies

In [126]:
def find_max_chained_nests(node): # for functions
    if isinstance(node, ast.Call):
        return 1 + max([find_max_chained_nests(arg) for arg in node.args]) if node.args else 0
    elif isinstance(node, ast.Attribute):
        return 1 + find_max_chained_nests(node.value)
    elif isinstance(node, ast.BinOp):
        left_nests = find_max_chained_nests(node.left) if hasattr(node, 'left') else 0
        right_nests = find_max_chained_nests(node.right) if hasattr(node, 'right') else 0
        return max(left_nests, right_nests)
    else:
        return 0

In [127]:
file_names = {}
num_functions = []
num_classes = []
num_variables = []
num_file_dependencies = []
num_dependencies = []
num_obj_nests = []
num_indents = []
total_lines = []

for f in file_list.keys():
    dependencies = []

    with open(f, 'r') as file:
        code = file.read()

    num_lines = code.count("\n")

    # Define a regular expression pattern to match multiline comments
    comment_pattern = re.compile(r'""".*?"""', re.MULTILINE | re.DOTALL)

    # Remove multiline comments from the code
    code_without_comments = re.sub(comment_pattern, '', code)

    # Split the code into lines and count indentation
    lines = code_without_comments.split('\n')
    max_indent = 0

    for line in lines:
        indent_level = len(line) - len(line.lstrip())
        max_indent = max(max_indent, indent_level)

    num_indents.append(max_indent // 4)
    total_lines.append(num_lines)

    # Use the ast module to parse the code and extract import statements
    tree = ast.parse(code)
    import_nodes = [node for node in ast.walk(tree) if isinstance(node, ast.Import) or isinstance(node, ast.ImportFrom)]

    # Extract dependencies from import statements
    
    filename_as_dep = f.replace("/home/dan/Documents/bnb_ai/", "").replace(".py", "").replace("/", ".")
    dep_count = 0

    for node in import_nodes:
        if isinstance(node, ast.Import):
            # Handle imports like: "import module"
            for alias in node.names:
                dependencies.append(alias.name)
                dep_count += 1
        elif isinstance(node, ast.ImportFrom):
            # Handle imports like: "from module import submodule"
            for alias in node.names:
                dependencies.append(node.module + "." + alias.name)
                dep_count += 1

    file_names[filename_as_dep] = dependencies
    
    # Count the number of functions, classes, and variables
    functions = [node.name for node in ast.walk(tree) if isinstance(node, ast.FunctionDef)]
    classes = [node.name for node in ast.walk(tree) if isinstance(node, ast.ClassDef)]
    variables = [node.targets[0].id for node in ast.walk(tree) if isinstance(node, ast.Assign) and isinstance(node.targets[0], ast.Name)]
    max_nests = max([find_max_chained_nests(node) for node in ast.walk(tree)], default=0)


    num_dependencies.append(dep_count)
    num_file_dependencies.append(len(import_nodes))
    num_functions.append(len(functions))
    num_classes.append(len(classes))
    num_variables.append(len(variables))
    num_obj_nests.append(max_nests)

# Create a DataFrame from the dependencies
df = pd.DataFrame([(index, value) for index, values in file_names.items() for value in values], columns=['File', 'Dependency'])
stat_df = pd.DataFrame({'Files':list(file_names.keys()), 'Functions': num_functions, 'Classes': num_classes, 
                        'Variables': num_variables, 'fileDependencies': num_file_dependencies, 
                        'Dependencies': num_dependencies, 'Lines': total_lines, 'Nests': num_obj_nests, 'Indents': num_indents})

In [128]:
stat_df

,Files,Functions,Classes,Variables,fileDependencies,Dependencies,Lines,Nests,Indents
0,app,1,0,2,5,12,40,1,2
1,views.profile,2,0,7,4,10,45,1,3
2,views.fav,2,0,6,6,9,65,1,4
3,views.add,10,0,13,6,17,270,2,4
4,views.main,7,0,9,7,13,178,1,4
5,views.post,4,0,13,5,12,118,2,5
6,views.authapi,3,0,5,2,7,60,1,2
7,views.payments,3,0,3,8,11,93,3,5
8,views.reviews,3,0,4,5,13,99,1,5
9,utils.firebase,14,0,39,10,12,332,3,4


In [129]:
df

,File,Dependency
0,app,os
1,app,config.initialize_app
2,app,dotenv.load_dotenv
3,app,flask.Flask
4,app,views.add
...,...,...
143,config.papertrail,logging.handlers.SysLogHandler
144,config.papertrail,os
145,config.redis,os
146,config.redis,flask_session.Session


get repo statistics

In [130]:
def get_num_libs():
    with open("requirements.txt", "r") as f:
        libs = f.readlines()
    return len(libs)


In [131]:
num_files = len(file_list.keys())
num_libs = get_num_libs() # pokud neni tak ses curak a mel bys si to pridat


FileNotFoundError: [Errno 2] No such file or directory: 'requirements.txt'

In [134]:
reduced_df = df
reduced_df.Dependency = reduced_df.Dependency.apply(lambda x: x.split(".")[0])
reduced_df.File = reduced_df.File.apply(lambda x: x.split(".")[0])
reduced_df = reduced_df.reset_index(drop=True).drop_duplicates()

visualize dependencies

In [135]:
from graphviz import Digraph
dot = Digraph(format='pdf')
dot.engine = 'dot'

# Create nodes for each unique dependency
unique_dependencies = reduced_df['Dependency'].unique().tolist()
for dep in unique_dependencies:
    dot.node(dep)

# Create edges between dependencies based on usage
for i, row in reduced_df.iterrows():
    dot.edge(row['File'], row['Dependency'])

dot.render("dependency_graph", view=True)

'dependency_graph.pdf'

In [ ]:
# TODO: odlisit zavislosti na knihovnach a vlastnich modulech